# Import Library

In [48]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, KFold

# Load Data

In [49]:
data = pd.read_csv('D:\Capstone Project\Deteksi_Cardiovascular\cardio.csv', sep=';')
data.head()

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Imam Bari Setiawan\AppData\Local\Temp\ipykernel_4880\1720964195.py:1: SyntaxWarning: invalid escape sequence '\C'
  data = pd.read_csv('D:\Capstone Project\Deteksi_Cardiovascular\cardio.csv', sep=';')


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


# Cleaning Data

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


In [51]:
data = data.drop(columns=['id'])

data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,70000.0,19468.865814,2467.251667,10798.0,17664.0,19703.0,21327.0,23713.0
gender,70000.0,1.349571,0.476838,1.0,1.0,1.0,2.0,2.0
height,70000.0,164.359229,8.210126,55.0,159.0,165.0,170.0,250.0
weight,70000.0,74.205690,14.395757,10.0,65.0,72.0,82.0,200.0
ap_hi,70000.0,128.817286,154.011419,-150.0,120.0,120.0,140.0,16020.0
ap_lo,70000.0,96.630414,188.472530,-70.0,80.0,80.0,90.0,11000.0
cholesterol,70000.0,1.366871,0.680250,1.0,1.0,1.0,2.0,3.0
gluc,70000.0,1.226457,0.572270,1.0,1.0,1.0,1.0,3.0
smoke,70000.0,0.088129,0.283484,0.0,0.0,0.0,0.0,1.0
alco,70000.0,0.053771,0.225568,0.0,0.0,0.0,0.0,1.0


In [52]:
def detect_outliers_iqr(data, features):
    outliers = []
    for feature in features:
        Q1 = data[feature].quantile(0.25)
        Q3 = data[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_indices = data[(data[feature] < lower_bound) | (data[feature] > upper_bound)].index
        outliers.extend(outlier_indices)
    return list(set(outliers))

# Fitur-fitur kontinu yang akan diidentifikasi outlier
continuous_features = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']

# Mendeteksi outlier dalam dataset
outliers_iqr = detect_outliers_iqr(data, continuous_features)

# Menampilkan baris data yang dianggap sebagai outlier
outliers_data = data.loc[outliers_iqr]
print(outliers_data)

         age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
32769  15251       2     180    71.0    160    110            1     1      0   
65539  22585       2     153   111.0    120     80            1     1      0   
4      17474       1     156    56.0    100     60            1     1      0   
9      19834       1     164    68.0    110     60            1     1      0   
65547  21221       1     163    88.0     90     60            1     1      0   
...      ...     ...     ...     ...    ...    ...          ...   ...    ...   
32733  21129       2     164    70.0     70    100            1     1      0   
32746  14562       2     167    65.0     90     60            1     1      1   
65516  18920       1     156    70.0    180     90            2     3      0   
32750  20259       1     152    53.0    152     53            1     1      0   
65523  21193       2     164    78.0    180     90            1     1      0   

       alco  active  cardio  
32769    

In [53]:
# Menghapus baris yang mengandung outlier dari dataframe
data2 = data.drop(outliers_iqr)

# Menampilkan dataframe setelah penghapusan outlier
print(data2)

         age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0      18393       2     168    62.0    110     80            1     1      0   
1      20228       1     156    85.0    140     90            3     1      0   
2      18857       1     165    64.0    130     70            3     1      0   
3      17623       2     169    82.0    150    100            1     1      0   
5      21914       1     151    67.0    120     80            2     2      0   
...      ...     ...     ...     ...    ...    ...          ...   ...    ...   
69993  19699       1     172    70.0    130     90            1     1      0   
69994  21074       1     165    80.0    150     80            1     1      0   
69995  19240       2     168    76.0    120     80            1     1      1   
69998  22431       1     163    72.0    135     80            1     2      0   
69999  20540       1     170    72.0    120     80            2     1      0   

       alco  active  cardio  
0        

In [54]:
# sum of duplicated values in the DataFrame
data2.duplicated(keep=False).sum()

48

In [55]:
data_cleaned = data2.drop_duplicates()

In [56]:
# Memeriksa nilai yang di luar dari 0 atau 1 pada fitur biner
binary_features = ['smoke', 'alco', 'active', 'cardio']
for feature in binary_features:
    outlier_indices = data_cleaned[~data_cleaned[feature].isin([0, 1])].index
    if len(outlier_indices) > 0:
        print(f"Outlier ditemukan pada fitur {feature}:")
        print(data_cleaned.loc[outlier_indices])
    else:
        print(f"Tidak ada outlier pada fitur {feature}.")

Tidak ada outlier pada fitur smoke.
Tidak ada outlier pada fitur alco.
Tidak ada outlier pada fitur active.
Tidak ada outlier pada fitur cardio.


In [57]:
data_cleaned.isnull().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [58]:
# Convert age from days to years
data_cleaned['age_years'] = round(data_cleaned['age'] / 365.25)

# Drop the original 'age' column
data_cleaned.drop(columns=['age'], inplace=True)

C:\Users\Imam Bari Setiawan\AppData\Local\Temp\ipykernel_4880\3984421960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['age_years'] = round(data_cleaned['age'] / 365.25)
C:\Users\Imam Bari Setiawan\AppData\Local\Temp\ipykernel_4880\3984421960.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=['age'], inplace=True)


In [59]:
data_cleaned.head()

,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years
0,2,168,62.0,110,80,1,1,0,0,1,0,50.0
1,1,156,85.0,140,90,3,1,0,0,1,1,55.0
2,1,165,64.0,130,70,3,1,0,0,0,1,52.0
3,2,169,82.0,150,100,1,1,0,0,1,1,48.0
5,1,151,67.0,120,80,2,2,0,0,0,0,60.0


In [60]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle

# Load your dataset (replace 'data_cleaned.csv' with your dataset file)
data_clean = pd.read_csv('data_cleaned.csv')

# Print the column names to check if 'age_years' is included
print("Column Names:", data.columns)

# Define your feature columns and target column
feature_columns = ['gender', 'age', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active']
target_column = 'cardio'

# Check if all feature columns exist in the DataFrame
missing_columns = [col for col in feature_columns if col not in data.columns]
if missing_columns:
    raise ValueError(f"Missing columns in the data: {missing_columns}")

# Extract features and target
X = data[feature_columns]
y = data[target_column]

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the feature data
scaler.fit(X)


Column Names: Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')


StandardScaler()

# Pemisahan Data Atribut dengan Label

In [61]:
X = data_cleaned.drop(columns='cardio', axis=1)
Y = data_cleaned['cardio']

In [62]:
print(X)

       gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  alco  \
0           2     168    62.0    110     80            1     1      0     0   
1           1     156    85.0    140     90            3     1      0     0   
2           1     165    64.0    130     70            3     1      0     0   
3           2     169    82.0    150    100            1     1      0     0   
5           1     151    67.0    120     80            2     2      0     0   
...       ...     ...     ...    ...    ...          ...   ...    ...   ...   
69993       1     172    70.0    130     90            1     1      0     0   
69994       1     165    80.0    150     80            1     1      0     0   
69995       2     168    76.0    120     80            1     1      1     0   
69998       1     163    72.0    135     80            1     2      0     0   
69999       1     170    72.0    120     80            2     1      0     0   

       active  age_years  
0           1       50.0

In [63]:
print(Y)

0        0
1        1
2        1
3        1
5        0
        ..
69993    1
69994    1
69995    0
69998    1
69999    0
Name: cardio, Length: 62478, dtype: int64


# Pisahkan Data Training dan Data Testing

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [65]:
print(X.shape, X_train.shape, X_test.shape)

(62478, 11) (49982, 11) (12496, 11)


# Model Training

In [66]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize the XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the model
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

# Evaluasi Model

In [67]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.729433418693982
Confusion Matrix:
[[4926 1397]
 [1984 4189]]
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.78      0.74      6323
           1       0.75      0.68      0.71      6173

    accuracy                           0.73     12496
   macro avg       0.73      0.73      0.73     12496
weighted avg       0.73      0.73      0.73     12496



In [70]:
# Assuming 'cleaned_df' is your existing cleaned DataFrame
data_cleaned.to_csv('data_cleaned.csv', index=False)

print("DataFrame saved to 'data_cleaned.csv'")


DataFrame saved to 'data_cleaned.csv'


In [71]:
input_data = (2,178,95.0,130,90,3,3,0,0,1,62.0)
input_data_as_numpy_array = np.array(input_data)
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

prediction = model.predict(input_data_reshaped)
print(prediction)

if (prediction[0]==1):
    print('Pasien terkena penyakit cardiovascular')
else:
    print('pasien tidak terkena penyakit cardiovascular')

[1]
Pasien terkena penyakit cardiovascular
